In [1]:
#Helpers to Run ML Algorithms

In [2]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
import datetime

/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
def fill_missing_mean(df_train, df_to_fill):
    '''
    A simple function that fills missing values of continuous columns 
        with the column median
    Inputs:
        df_train (df): the training df. Function computes means from this value AND fills this 
        df_to_fill (df): the df whose continuous NAs should be filled 
    Returns:
        df_train (df): the training dataset
        df_to_fill: the testing dataset with its data filled by the training data median

    '''
    df_train_num = df_train.select_dtypes(include=[np.number])
    means = df_train_num.mean().to_dict()
    
    df_train = df_train.fillna(value=means)
    df_to_fill = df_to_fill.fillna(value=means)
    print("Finished filling NAs with mean...")
    return df_train, df_to_fill

In [4]:
def normalize_continuous(df, scaler = None):
    '''
    A simple function that normalizes the values of of continuous columns 
        using data from the training set
    Inputs:
        df (df): either the training or the testing df
        scaler: the scaler object. It will be None for training and exist for testing 
    Returns:
        df (df): the standardized df
        scaler: the scaler object
    '''
    if scaler is None: #Training case
        scaler = sk.preprocessing.StandardScaler() #Set up scaler
        df_num = df.select_dtypes(include=[np.number]) #find numeric columns
        df_num_scaled = scaler.fit_transform(df_num) #Normalize them
        df_num_cols = list(df_num.columns) 
        df.loc[:, df_num_cols] = df_num_scaled #Insert columns back into the main df 
        print("Finished normalizing training data")
    else: #Testing case
        df_num = df.select_dtypes(include=[np.number]) #find numeric columns
        df_num_scaled = scaler.transform(df_num) #Normalize them
        df_num_cols = list(df_num.columns) 
        df.loc[:, df_num_cols] = df_num_scaled #Insert columns back into the main df 
        print("Finished normalizing test data...")
    return df, scaler

In [5]:
def one_hot_encode(df, cat_vars): 
    '''
    A function to one-hot encode given categorical variables
    Inputs:
        df (df): a pandas dataframe
        cat_vars (list of strings): a list of the categorical variables to one-hot encode
    '''

    df = pd.get_dummies(df, columns = cat_vars)
    print("finished one-hot encoding...")
    return df

In [6]:
def standardize_columns(train, test):
    '''
    A function to ensure that training and testing data have identical columns
    after one-hot encoding
    If a column is in training but not testing, adds a column of 0s to testing
    If column is in testing but not training, it is removed
    Inputs:
        train (df): the training df
        test (df): the testing df
    Outputs:
        train, test (df): the datasets with identical columns
    '''
    train_cols = list(train.columns)
    test_cols = list(test.columns)
    
    for tr_col in train_cols:
        if tr_col not in test_cols:
            test[tr_col] = 0
    
    for test_col in test_cols:
        if test_col not in train_cols:
            test = test.drop(test_col, axis=1)
    print("finished standardizing...")
    return (train, test)

In [ ]:
def split_train_test_by_year(df, y, test_year, num_years):
    '''
    isin syntax from: https://www.kite.com/python/answers/how-to-filter-a-pandas-dataframe-with-a-list-by-%60in%60-or-%60not-in%60-in-python
    Inputs:
        df (df): the dataframe with both the X and y 
        y (string): the column name in the df that is the target
        test_year(int): the year we seek to predict in 2016-2020
        num_years(int): the number of years to be included in the training set

    Output:
    train_df (df): A training dataframe
    test_X (df): Testing df of predictors
    test_y (df): Testing target
    '''
    year_range = np.arange(test_year - num_years, test_year)
    train_filter = df.Year.isin(year_range)
    train_df = df[train_filter]
    train_X = train_df.drop(columns=[y])
    train_y = train_df[y]
    
    test_df = df[df.Year==test_year]
    test_X = test_df.drop(columns = [y], inplace=True)
    test_y = test_df[y]
    print("finished splitting by year...")
    return train_X, train_y, test_X, test_y

In [24]:
def prep_data(df, y, test_year, num_years, vars_to_onehot):
    '''
    Helper function that aggregates the above helpers to prepare for imputation in 
    an ML algorithm. Specifically this:
        Splits the training set and testing set based on year 
            using split_train_test_by_year
        One-hot encodes and standardizes the columns using 
            one_hot_encode and standardize_column
        Normalizes all continuous variables using normalize_continuous
    Inputs:
    df (pandas DataFrame): the dataframe with training and testing data, predictors and target
    y (string): the name of the target column
    num_years (int): the number of years to be included in the training set
    test_year (int): the year we seek to predict 2015_2020
    
    Outputs:
    train_df (df): a standardized training set with one-hot encorded categorical columns
    test_df (df): the test dataframe, again standardized as above
    test_y (Series): the test target
    '''
    train_df, train_y, test_df, test_y = split_train_test_by_year(df, y, test_year, num_years)
    train_df, test_df = standardize_columns(train_df, test_df)
    train_df, test_df = fill_missing_mean(train_df, test_df)
    train_df, scaler = normalize_continuous(train_df)
    test_df, doesnt_matter = normalize_continuous(test_df, scaler)
    train_df = one_hot_encode(train_df, vars_to_onehot)
    test_df = one_hot_encode(test_df, vars_to_onehot)
    train_df, test_df = standardize_columns(train_df, test_df)
    return train_df, train_y, test_df, test_y
    

In [36]:
data = pd.read_csv("../intermediate_data/df_2015_to_present.csv")

In [39]:
def convert_to_categorical(df, cols_to_convert):
    '''
    Convert columns to categorical
    Inputs:
        df (pd.DataFrame): The Pandas df
        cols_to_convert (list of strings): The columns to convert    
    Output:
        df - the updated dataframe
    '''
    for col in cols_to_convert:
        df[col]=df[col].astype("category")

    return df

data = convert_to_categorical(data, ["Beat", "Year", "Month", "Watch"])


In [11]:
train_df, train_y, test_df, test_y = split_train_test_by_year(data, "Arrest", 2017, 2)



finished splitting by year...


In [40]:
#Test the code! 
train_X, train_y, test_X, test_y = prep_data(data, "Arrest", 2017, 2, 
                                       ["Year", "Month", "Week", "Beat"])

finished splitting by year...
finished standardizing...
Finished filling NAs with mean...
Finished normalizing training data
Finished normalizing test data...
finished one-hot encoding...
finished one-hot encoding...
finished standardizing...


In [41]:
train_X.describe() #Looking fresh :) 

,ID,Day,Hour,PRCP,SNOW,TMAX,TMIN,count_l_stops,count_bus_stops,count_metra_stops,...,Beat_2521,Beat_2522,Beat_2523,Beat_2524,Beat_2525,Beat_2531,Beat_2532,Beat_2533,Beat_2534,Beat_2535
count,5.340900e+05,5.340900e+05,5.340900e+05,5.340900e+05,5.340900e+05,5.340900e+05,5.340900e+05,5.340900e+05,5.340900e+05,5.340900e+05,...,534090.000000,534090.000000,534090.000000,534090.000000,534090.000000,534090.000000,534090.00000,534090.000000,534090.000000,534090.000000
mean,-5.833968e-15,-2.426935e-14,1.807149e-14,-6.361288e-14,-1.527197e-14,-1.510259e-14,1.499653e-13,-8.963044e-14,-1.005280e-15,-7.794098e-15,...,0.004338,0.003073,0.002887,0.002558,0.002273,0.003479,0.00454,0.005991,0.005198,0.003763
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,0.065722,0.055345,0.053655,0.050508,0.047622,0.058879,0.06723,0.077173,0.071907,0.061231
min,-1.796652e+01,-1.653279e+00,-1.993694e+00,-3.779970e-01,-1.358532e-01,-2.747795e+00,-2.873175e+00,-5.721857e-01,-2.123496e+00,-4.499297e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,-3.557260e-01,-8.633741e-01,-6.294804e-01,-3.779970e-01,-1.358532e-01,-7.970072e-01,-7.117863e-01,-5.721857e-01,-7.038846e-01,-4.499297e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,3.738473e-02,3.937406e-02,1.284163e-01,-3.779970e-01,-1.358532e-01,1.783866e-01,2.624904e-02,-5.721857e-01,-3.063934e-01,-4.499297e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,4.263588e-01,8.292787e-01,8.863130e-01,-2.190015e-01,-1.358532e-01,8.611624e-01,9.224348e-01,6.704085e-01,5.453734e-01,-4.499297e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
max,3.472606e+00,1.732027e+00,1.492630e+00,1.192825e+01,2.425735e+01,1.543938e+00,1.555037e+00,4.398191e+00,3.441381e+00,5.954221e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
